In [1]:
#Imports
import regex as re


In [5]:
sample_text = """Petitioner John Angus Smith and his companion went from Tennessee to Florida to buy cocaine; they hoped to resell it at a profit. While in Florida, they met petitioner's acquaintance, Deborah Hoag. Hoag agreed to, and in fact did, purchase cocaine for petitioner.\
    She then accompanied petitioner and his friend to her motel room, where they were joined by a drug dealer. While Hoag listened, petitioner and the dealer discussed petitioner's MAC–10 firearm, which had been modified to operate as an automatic. The MAC–10 apparently is a \
    favorite among criminals. It is small and compact, lightweight, and can be equipped with a silencer. Most important of all, it can be devastating: A fully automatic MAC–10 can fire more than 1,000 rounds per minute. The dealer expressed his interest in becoming the owner \
    of a MAC–10, and petitioner promised that he would discuss selling the gun if his arrangement with another potential buyer fell through. Unfortunately for petitioner, Hoag had contacts not only with narcotics traffickers but also with law enforcement officials. \
    In fact, she was a confidential informant. Consistent with her post, she informed the Broward County Sheriff's Office of petitioner's activities. The Sheriff's Office responded quickly, sending an undercover officer to Hoag's motel room. Several others were assigned to \
    keep the motel under surveillance. Upon arriving at Hoag's motel room, the undercover officer presented himself to petitioner as a pawnshop dealer. Petitioner, in turn, presented the officer with a proposition: He had an automatic MAC–10 and silencer with which he might \
    be willing to part. Petitioner then pulled the MAC–10 out of a black canvas bag and showed it to the officer. The officer examined the gun and asked petitioner what he wanted for it. Rather than asking for money, however, petitioner asked for drugs. He was willing to trade his MAC–10, he said, \
    for two ounces of cocaine. The officer told petitioner that he was just a pawnshop dealer and did not distribute narcotics. Nonetheless, he indicated that he wanted the MAC–10 and would try to get the cocaine. The officer then left, promising to return within an hour."""

In [4]:
#Creating a byte tokenizer
def byte_tokenizer(text):
    ''' 
    Maps a string to a list of byte tokens
    '''
    tokens = text.encode('utf-8')
    tokens = list(map(int, tokens))
    return tokens

In [5]:
#Function to introduce the byte-pair encoding
def get_pairs(bts):
    """ 
    Returns a dictionary of all byte pairs in the input and their frequency
    """
    counts = {}
    for pair in zip(bts, bts[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [6]:
#Function to merge the most frequent pair
def merge_pairs(bts, pair, idx):
    """ Merges the most frequent pair in the input
    Args:
        bts: list of byte tokens
        pair: pair to merge
        idx: new byte token to replace the pair
    Returns:
        new_bts: list of byte tokens with the most frequent pair merged
    """
    new_bts = []
    i = 0
    while i < len(bts):
        if i < len(bts) - 1 and bts[i] == pair[0] and bts[i + 1] == pair[1]:
            new_bts.append(idx)
            i += 2
        else:
            new_bts.append(bts[i])
            i += 1
    return new_bts

In [7]:
sample_tokens = byte_tokenizer(sample_text)

In [8]:
sample_tokens[:5]

[80, 101, 116, 105, 116]

In [8]:
#First try of an implementation of the BPE algorithm for 20 merges
#Hyperparameters
vocab_size = 276
num_merges = vocab_size - 256
bts = list(sample_tokens)

#Main loop
merges = {}
for i in range(num_merges):
    pairs = get_pairs(bts)
    if not pairs:
        break
    best_pair = max(pairs, key=pairs.get)
    btx = 256 + i
    print('##-------------------------------------------------##')
    print(f'  Merging pair {best_pair} into new byte token {btx}')
    bts = merge_pairs(bts, best_pair, btx)
    merges[best_pair] = btx


##-------------------------------------------------##
  Merging pair (101, 114) into new byte token 256
##-------------------------------------------------##
  Merging pair (101, 32) into new byte token 257
##-------------------------------------------------##
  Merging pair (32, 97) into new byte token 258
##-------------------------------------------------##
  Merging pair (116, 105) into new byte token 259
##-------------------------------------------------##
  Merging pair (100, 32) into new byte token 260
##-------------------------------------------------##
  Merging pair (116, 104) into new byte token 261
##-------------------------------------------------##
  Merging pair (32, 32) into new byte token 262
##-------------------------------------------------##
  Merging pair (111, 110) into new byte token 263
##-------------------------------------------------##
  Merging pair (105, 110) into new byte token 264
##-------------------------------------------------##
  Merging pair (

In [103]:
#GPT-4 style BPE tokenizer
class BasicTokenizer:

    def __init__(self):
        self.tokens = []
        self.merges = {}

    def get_pairs(self, bts):
        counts = {}
        for pair in zip(bts, bts[1:]):
            counts[pair] = counts.get(pair, 0) + 1
        return counts
    
    def merge_pairs(self, bts, pair, idx):
        new_bts = []
        i = 0
        while i < len(bts):
            if i < len(bts) - 1 and bts[i] == pair[0] and bts[i + 1] == pair[1]:
                new_bts.append(idx)
                i += 2
            else:
                new_bts.append(bts[i])
                i += 1
        return new_bts
    
            
    def train(self, text, vocab_size, verbose=False):
        ''' Train the tokenizer on the given text. '''
        self.tokens = list(map(int, text.encode('utf-8')))
        tokens = self.tokens
        num_merges = vocab_size - 256
        for i in range(num_merges):
            pairs = self.get_pairs(tokens)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            btx = 256 + i
            if verbose:
                print(f'Merging pair {best_pair} into new byte token {btx}')
            tokens = self.merge_pairs(tokens, best_pair, btx)
            self.merges[best_pair] = btx
            

    def encode(self, text):
        ''' Encode the text using the learned merges. '''
        bts = list(map(int, text.encode('utf-8')))
        for i in range(len(bts)):
            for k, v in self.merges.items():
                if i < len(bts) - 1 and bts[i] == k[0] and bts[i + 1] == k[1]:
                    bts[i] = v
                    bts.pop(i + 1)
        return bts

    def decode(self, ids):
        ''' Decode the ids using the learned merges. '''
        ids = list(ids)
        for i in ids:
            for k, v in self.merges.items():
                if i == v:
                    new_i = [k[0], k[1]]
                    index = ids.index(i)
                    ids.pop(index)
                    ids[index:index] = new_i
        text = ''.join(map(chr, ids))
        return text

In [104]:
tokenizer = BasicTokenizer()
vocab_size = 276
tokenizer.train(sample_text, vocab_size, verbose=True)

Merging pair (101, 114) into new byte token 256
Merging pair (101, 32) into new byte token 257
Merging pair (32, 97) into new byte token 258
Merging pair (116, 105) into new byte token 259
Merging pair (100, 32) into new byte token 260
Merging pair (116, 104) into new byte token 261
Merging pair (32, 32) into new byte token 262
Merging pair (111, 110) into new byte token 263
Merging pair (105, 110) into new byte token 264
Merging pair (256, 32) into new byte token 265
Merging pair (101, 260) into new byte token 266
Merging pair (115, 32) into new byte token 267
Merging pair (116, 32) into new byte token 268
Merging pair (101, 110) into new byte token 269
Merging pair (46, 32) into new byte token 270
Merging pair (44, 32) into new byte token 271
Merging pair (116, 111) into new byte token 272
Merging pair (258, 110) into new byte token 273
Merging pair (259, 263) into new byte token 274
Merging pair (111, 114) into new byte token 275


In [107]:
new_sample = '''Copy paste of the Wikipedia article on Taylor Swift, as of Feb 16, 2024.
---

Main menu

WikipediaThe Free Encyclopedia

Search
Create account
Log in

Personal tools
Contents  hide
(Top)
Life and career
Toggle Life and career subsection
Artistry
Toggle Artistry subsection'''
encoded = tokenizer.encode(new_sample)

In [108]:
text = tokenizer.decode(encoded)
text

'Copy paste of the Wikipedia article on Taylor Swift, as of Feb 16, 2024.\n---\n\nMain menu\n\nWikipediaThe Free Encyclopedia\n\nSearch\nCreate account\nLog in\n\nPersonal tools\nContents  hide\n(Top)\nLife and career\nToggle Life and careĀ subsection\nArtistry\nToggle Artistry subsection'

In [ ]:
class RegexTokenizer(BasicTokenizer):

    def __init__(self):
        super().__init__()
        #This is the GPT-4 regex pattern
        self.pattern = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

    def train(self, text, vocab_size, verbose=False):
        ''' Train the tokenizer on the given text. '''
        